In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer

full_data = pd.read_csv('https://dso-560-nlp-text-analytics.s3.amazonaws.com/Full+data.csv')
extra_data = pd.read_csv('https://dso-560-nlp-text-analytics.s3.amazonaws.com/extra_data.csv')
sme_tag = pd.read_excel('https://dso-560-nlp-text-analytics.s3.amazonaws.com/USC+Product+Attribute+Data+03302020.xlsx')
additional_tag = pd.read_csv('https://dso-560-nlp-text-analytics.s3.amazonaws.com/usc_additional_tags.csv')
#attributes =  pd.read_excel('Womens+Attributes.xlsx')

/Users/dwko/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
full_data = full_data.iloc[:,:14]
full_data.head()

,product_id,brand,mpn,name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,tsv,bc_product_id
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,FILA,400010319073,Original Fitness Sneakers,Vintage Fitness leather sneakers with logo pri...,TheMensStore/Shoes/Sneakers/LowTop,2019-11-15 23:36:38.98161+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/fila-original-...,Leather/synthetic upper\nLace-up closure\nText...,"{""Needs Attributes""}","'design':12 'fila':1A 'fit':3A,6 'leather':7 '...",NaN
1,01DSQXJBX0R7DCW7KTAC1SW547,CHANEL,400011497371,HAT,NaN,Unknown,2019-11-15 16:15:34.809725+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/chanel-hat/pro...,WOOL TWEED & FELT,"{""Needs Review""}",'chanel':1A 'hat':2A,NaN
2,01DPGV8TGRAB993PF7Z3YWG2VR,Frame,LWBT0099,Petit Oval Buckle Belt,A Timeless Leather Belt Crafted From Smooth Co...,Accessories,2019-10-06 15:33:38.451504+00,2019-12-19 20:40:30.786144+00,NaN,https://frame-store.com/products/petit-oval-bu...,NaN,{},"'belt':5A,9 'buckl':4A,21 'cowhid':13 'craft':...",NaN
3,01DSR8G3F7DBRTMP8THF97XSQ2,Lilly Pulitzer Kids,400011844797,Little Gir's & Girl's Ariana One-Piece UPF 50+...,Pretty ruffle sleeves and trim elevate essenti...,"JustKids/Girls214/Girls/SwimwearCoverups,JustK...",2019-11-15 19:26:34.947426+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/lilly-pulitzer...,Scoopneck\nAdjustable straps\nFlutter sleeves\...,"{""Needs Review""}",'50':14A 'allov':28 'ariana':9A 'color':27 'el...,NaN
4,01DSR8G5GP519DEDCSKBMWQVK5,Kissy Kissy,400094474585,Baby Girl's Endearing Elephants Pima Cotton Co...,Versatile convertible gown with elephant applique,JustKids/Baby024months/InfantGirls/FootiesRompers,2019-11-15 19:26:36.995998+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/kissy-kissy-ba...,V-neckline\nLong sleeves\nFront snap closure\n...,"{""Needs Review""}","'appliqu':17 'babi':3A 'convert':10A,13 'cotto...",NaN


In [3]:
extra_data.tail()

,product_id,mpn,brand,brand_category,name,labels,updated_at,bc_product_id,saleable,details,created_at,brand_canonical_url,notes,description
6616,01E2KZ045BCT1Q2Z2PBC0CWRGH,5441202,ATM ANTHONY THOMAS MELILLO,Unknown,Micro Twill Pull On Pants,[],2020-03-26 00:06:42.342000+00:00,1957,2020-03-25 16:34:08.933000+00:00,"True to size. XS=0, S=2-4, M=6-8, L=10. High r...",2020-03-04 23:44:52.899000+00:00,https://shop.nordstrom.com/s/atm-anthony-thoma...,NaN,"These casual trousers are cut for an easy, dra..."
6617,01E2KXZ0WNQ96S64TDZE3WB7CX,5504588,ATM ANTHONY THOMAS MELILLO,Unknown,Brushed Twill Crop Wide Leg Pants,[],2020-04-02 23:13:56.436000+00:00,1801,2020-04-02 23:13:56.436000+00:00,True to size. High rise.,2020-03-04 23:26:48.207000+00:00,https://shop.nordstrom.com/s/atm-anthony-thoma...,NaN,Brushed twill gives these cropped wide-leg pan...
6618,01E2KXWWY6JAGENED7P62090NP,4496777,ATM ANTHONY THOMAS MELILLO,Unknown,Slim Crop Pants,[],2020-04-02 23:11:55.640000+00:00,1791,2020-04-02 23:11:55.640000+00:00,Mid rise. True to size.,2020-03-04 23:25:38.623000+00:00,https://shop.nordstrom.com/s/atm-anthony-thoma...,NaN,"Tailored from a soft, dense knit with cropped,..."
6619,01E2KZTEPE7QK1V8THASF7HWNR,5289605,ATM ANTHONY THOMAS MELILLO,Unknown,Camo Print Silk Skirt,[],2020-04-02 23:39:04.835000+00:00,2077,2020-04-02 23:39:04.835000+00:00,"True to size. XS=0, S=2-4, M=6-8, L=10.",2020-03-04 23:59:15.654000+00:00,https://shop.nordstrom.com/s/atm-anthony-thoma...,NaN,Flatlock-stitched bias seams enhance the fluid...
6620,01E2KY2T6YV7R4VXT4BVV4TWT5,4788992,ATM ANTHONY THOMAS MELILLO,Unknown,Stretch Pima Cotton Baby Tee,[],2020-04-02 23:33:25.438000+00:00,1818,2020-04-02 23:33:25.438000+00:00,"True to size. XS=0, S=2-4, M=6-8, L=10.",2020-03-04 23:28:52.433000+00:00,https://shop.nordstrom.com/s/atm-anthony-thoma...,NaN,"Made from ATM’s signature Peruvian jersey, thi..."


In [4]:
# Adding extra_data to full_data and removing the duplicates with same product_id

clean_df = pd.concat([full_data, extra_data], ignore_index=True)
clean_df.drop_duplicates(subset= ['product_id'], keep = 'first', inplace=True)
clean_df = clean_df[list(full_data.columns)]
clean_df.head()

/Users/dwko/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,product_id,brand,mpn,name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,tsv,bc_product_id
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,FILA,400010319073,Original Fitness Sneakers,Vintage Fitness leather sneakers with logo pri...,TheMensStore/Shoes/Sneakers/LowTop,2019-11-15 23:36:38.98161+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/fila-original-...,Leather/synthetic upper\nLace-up closure\nText...,"{""Needs Attributes""}","'design':12 'fila':1A 'fit':3A,6 'leather':7 '...",NaN
1,01DSQXJBX0R7DCW7KTAC1SW547,CHANEL,400011497371,HAT,NaN,Unknown,2019-11-15 16:15:34.809725+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/chanel-hat/pro...,WOOL TWEED & FELT,"{""Needs Review""}",'chanel':1A 'hat':2A,NaN
2,01DPGV8TGRAB993PF7Z3YWG2VR,Frame,LWBT0099,Petit Oval Buckle Belt,A Timeless Leather Belt Crafted From Smooth Co...,Accessories,2019-10-06 15:33:38.451504+00,2019-12-19 20:40:30.786144+00,NaN,https://frame-store.com/products/petit-oval-bu...,NaN,{},"'belt':5A,9 'buckl':4A,21 'cowhid':13 'craft':...",NaN
3,01DSR8G3F7DBRTMP8THF97XSQ2,Lilly Pulitzer Kids,400011844797,Little Gir's & Girl's Ariana One-Piece UPF 50+...,Pretty ruffle sleeves and trim elevate essenti...,"JustKids/Girls214/Girls/SwimwearCoverups,JustK...",2019-11-15 19:26:34.947426+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/lilly-pulitzer...,Scoopneck\nAdjustable straps\nFlutter sleeves\...,"{""Needs Review""}",'50':14A 'allov':28 'ariana':9A 'color':27 'el...,NaN
4,01DSR8G5GP519DEDCSKBMWQVK5,Kissy Kissy,400094474585,Baby Girl's Endearing Elephants Pima Cotton Co...,Versatile convertible gown with elephant applique,JustKids/Baby024months/InfantGirls/FootiesRompers,2019-11-15 19:26:36.995998+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/kissy-kissy-ba...,V-neckline\nLong sleeves\nFront snap closure\n...,"{""Needs Review""}","'appliqu':17 'babi':3A 'convert':10A,13 'cotto...",NaN


In [5]:
len(clean_df)

48087

In [6]:
# I removed columns that would not be helpful for the text analytics (datetime)
train = clean_df.drop(columns=['created_at', 'updated_at', 'deleted_at', 'bc_product_id'])


# I made the entire dataframe in lowercase
for columns in train.columns:
    train[columns] = train[columns].str.lower() 
    
train.head()

,product_id,brand,mpn,name,description,brand_category,brand_canonical_url,details,labels,tsv
0,01dsrpsztdw2pgk1ywyxjgkzz0,fila,400010319073,original fitness sneakers,vintage fitness leather sneakers with logo pri...,themensstore/shoes/sneakers/lowtop,https://www.saksfifthavenue.com/fila-original-...,leather/synthetic upper\nlace-up closure\ntext...,"{""needs attributes""}","'design':12 'fila':1a 'fit':3a,6 'leather':7 '..."
1,01dsqxjbx0r7dcw7ktac1sw547,chanel,400011497371,hat,NaN,unknown,https://www.saksfifthavenue.com/chanel-hat/pro...,wool tweed & felt,"{""needs review""}",'chanel':1a 'hat':2a
2,01dpgv8tgrab993pf7z3ywg2vr,frame,lwbt0099,petit oval buckle belt,a timeless leather belt crafted from smooth co...,accessories,https://frame-store.com/products/petit-oval-bu...,NaN,{},"'belt':5a,9 'buckl':4a,21 'cowhid':13 'craft':..."
3,01dsr8g3f7dbrtmp8thf97xsq2,lilly pulitzer kids,400011844797,little gir's & girl's ariana one-piece upf 50+...,pretty ruffle sleeves and trim elevate essenti...,"justkids/girls214/girls/swimwearcoverups,justk...",https://www.saksfifthavenue.com/lilly-pulitzer...,scoopneck\nadjustable straps\nflutter sleeves\...,"{""needs review""}",'50':14a 'allov':28 'ariana':9a 'color':27 'el...
4,01dsr8g5gp519dedcskbmwqvk5,kissy kissy,400094474585,baby girl's endearing elephants pima cotton co...,versatile convertible gown with elephant applique,justkids/baby024months/infantgirls/footiesrompers,https://www.saksfifthavenue.com/kissy-kissy-ba...,v-neckline\nlong sleeves\nfront snap closure\n...,"{""needs review""}","'appliqu':17 'babi':3a 'convert':10a,13 'cotto..."


In [7]:
# Combining the sme_tag and additional_tag
tag = pd.concat([sme_tag, additional_tag], ignore_index=True)
for columns in tag.columns:
    tag[columns] = tag[columns].str.lower() 

#the attribute name/vaue from sme_tag included punctuations including spaces, but additional_tag stored the data in one chunk of words
# ex) additonal color vs additionalcolor

# I decided to follow the additional_tag format by removing all punctuations and spaces for easier analysis moving on
remove = [' ', '_', '(', ')', '-', ',', '&', '"', '"', '/']
for i in remove:
    tag['attribute_name'] = tag['attribute_name'].str.replace(i, '')
    tag['attribute_value'] = tag['attribute_value'].str.replace(i, '')

tag.drop_duplicates(keep='first', inplace=True) #only removed duplicates that have same data in all of the columns
tag.reset_index(inplace = True)

In [8]:
tag['product_id'].nunique()

3970

In [9]:
# For the purpose of the project, we will only choose 4 out of these attribute names
attribute_name = list(tag.attribute_name.unique())
attribute_name.sort()
attribute_name

['additionalcolor',
 'beltbucklematerial',
 'beltbuckleshape',
 'beltclosure',
 'beltmaterial',
 'beltwidth',
 'calfwidth',
 'category',
 'classbelts',
 'classblazerscoatsandjackets',
 'classbooties',
 'classboots',
 'classdress',
 'classflats',
 'classhandbags',
 'classjumpsuitandromper',
 'classmulesandslides',
 'classpantsandleggings',
 'classpumpsandheels',
 'classsandals',
 'classshorts',
 'classskirts',
 'classslippers',
 'classsneakersandathletic',
 'classsunglasses',
 'classwedges',
 'closureblazerscoatsandjackets',
 'closurehandbag',
 'closureonepiece',
 'closurepantsandleggings',
 'closureshoe',
 'closureshorts',
 'closureskirts',
 'closuresweater',
 'closuretop',
 'color',
 'drycleanonly',
 'embellishment',
 'fit',
 'gender',
 'heelheight',
 'heelshape',
 'legstyle',
 'legstylejeans',
 'lengthblazers',
 'lengthblazerscoatsandjackets',
 'lengthcoatsandjackets',
 'lengthjeans',
 'lengthonepiece',
 'lengthpantsandleggings',
 'lengthshorts',
 'lengthskirts',
 'lengthtop',
 'mate

In [10]:
attribute_value = list(tag.attribute_value.unique())
attribute_value.sort()
attribute_value

['100%cashmere',
 '100%cotton',
 '100%linen',
 '100%silk',
 '100%wool',
 '14zip',
 '5pocketpantnondenim',
 'abstract',
 'accessory',
 'acetate',
 'acid',
 'acrylic',
 'active',
 'activetrack',
 'adjustable',
 'aline',
 'alpaca',
 'androgynous',
 'animal',
 'ankle',
 'anklestrap',
 'anorakraincoatwindbreaker',
 'asymmetrical',
 'athleisure',
 'aviatorsquare',
 'aviatorteardrop',
 'backless',
 'backpacks',
 'backtie',
 'backzip',
 'backzipper',
 'baggy',
 'ballet',
 'bandcollar',
 'beachbags',
 'beaded',
 'beiges',
 'belt',
 'beltbagsfannypack',
 'belted',
 'belts',
 'bermuda',
 'bike',
 'black',
 'blacks',
 'blazer',
 'blazerdress',
 'blazerscoatsjackets',
 'block',
 'blouse',
 'blues',
 'boatneck',
 'bodycon',
 'bodysuit',
 'boho',
 'bombervarsity',
 'bootcut',
 'bootie',
 'booties',
 'boots',
 'bottom',
 'boyfriend',
 'brass',
 'browns',
 'bucketbags',
 'buckle',
 'buckles',
 'burgundies',
 'businesscasual',
 'businesscasualdress',
 'bustier',
 'buttondown',
 'buttonedback',
 'buttone

# Out of all the attributes above, we are only going to use: 
- **style** 
- **occasion** 
- **category** 
- **fit**

In [11]:
# Creating new columns for the targeted categories
train['style'] = ""
train['occasion'] = ""
train['category'] = ""
train['fit'] = ""

In [12]:
group_tag = tag.groupby('product_id')
tagged_product = list(group_tag.groups.keys())

In [13]:
group_tag.get_group('01e2fdnd1xhvrj8p3whhkjgg1x')

,index,product_id,product_color_id,attribute_name,attribute_value
38797,45259,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,uppermaterial,suede
38798,45260,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,primarycolor,blacks
38799,45261,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,style,casual
38800,45262,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,style,boho
38801,45263,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,style,modern
95680,116601,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,heelheight,medium
95681,116602,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,toestyle,pointedtoe
95682,116603,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,category,shoe
95683,116604,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,shaftheight,midcalf
95684,116605,01e2fdnd1xhvrj8p3whhkjgg1x,01e2fdnd27n1jekagccgmn7ngt,heelshape,cone


#### This code will loop through each of the product_id from the tag.
#### Using the dataframe above, if the attribute_name includes any of the 4 focus attributes ('style', 'occasion', 'category', 'fit'), the code will add the corresponding attribute_value to the new columns created in the full_data + extra_data.

In [14]:
focus_attribute = ['style', 'occasion', 'category', 'fit']

for x in tagged_product:
    product = group_tag.get_group(x)
    focus = product.loc[product['attribute_name'].isin(focus_attribute)].reset_index()
    for i in range(0,len(focus)):
        if focus.loc[i, 'attribute_name'] == 'style':
            train.loc[train['product_id'] == x, 'style'] += (focus.loc[i, 'attribute_value'] + " ")
        elif focus.loc[i, 'attribute_name'] == 'occasion':
            train.loc[train['product_id'] == x, 'occasion'] += (focus.loc[i, 'attribute_value'] + " ")
        elif focus.loc[i, 'attribute_name'] == 'category':
            train.loc[train['product_id'] == x, 'category'] += (focus.loc[i, 'attribute_value'] + " ")
        elif focus.loc[i, 'attribute_name'] == 'fit':
            train.loc[train['product_id'] == x, 'fit'] += (focus.loc[i, 'attribute_value'] + " ")

In [15]:
# Removing duplicated words in a cell for the 4 columns
# source: https://stackoverflow.com/questions/47316783/python-dataframe-remove-duplicate-words-in-the-same-cell-within-a-column-in-pyt

for i in focus_attribute:
    train[i] = (train[i].str.split()
                              .apply(lambda x: OrderedDict.fromkeys(x).keys())
                              .str.join(' '))

The result for the 2 lines above can be observed here.

In [16]:
train

,product_id,brand,mpn,name,description,brand_category,brand_canonical_url,details,labels,tsv,style,occasion,category,fit
0,01dsrpsztdw2pgk1ywyxjgkzz0,fila,400010319073,original fitness sneakers,vintage fitness leather sneakers with logo pri...,themensstore/shoes/sneakers/lowtop,https://www.saksfifthavenue.com/fila-original-...,leather/synthetic upper\nlace-up closure\ntext...,"{""needs attributes""}","'design':12 'fila':1a 'fit':3a,6 'leather':7 '...",,,,
1,01dsqxjbx0r7dcw7ktac1sw547,chanel,400011497371,hat,NaN,unknown,https://www.saksfifthavenue.com/chanel-hat/pro...,wool tweed & felt,"{""needs review""}",'chanel':1a 'hat':2a,,,,
2,01dpgv8tgrab993pf7z3ywg2vr,frame,lwbt0099,petit oval buckle belt,a timeless leather belt crafted from smooth co...,accessories,https://frame-store.com/products/petit-oval-bu...,NaN,{},"'belt':5a,9 'buckl':4a,21 'cowhid':13 'craft':...",,,,
3,01dsr8g3f7dbrtmp8thf97xsq2,lilly pulitzer kids,400011844797,little gir's & girl's ariana one-piece upf 50+...,pretty ruffle sleeves and trim elevate essenti...,"justkids/girls214/girls/swimwearcoverups,justk...",https://www.saksfifthavenue.com/lilly-pulitzer...,scoopneck\nadjustable straps\nflutter sleeves\...,"{""needs review""}",'50':14a 'allov':28 'ariana':9a 'color':27 'el...,,,,
4,01dsr8g5gp519dedcskbmwqvk5,kissy kissy,400094474585,baby girl's endearing elephants pima cotton co...,versatile convertible gown with elephant applique,justkids/baby024months/infantgirls/footiesrompers,https://www.saksfifthavenue.com/kissy-kissy-ba...,v-neckline\nlong sleeves\nfront snap closure\n...,"{""needs review""}","'appliqu':17 'babi':3a 'convert':10a,13 'cotto...",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48989,01e2kz045bct1q2z2pbc0cwrgh,atm anthony thomas melillo,5441202,micro twill pull on pants,"these casual trousers are cut for an easy, dra...",unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"true to size. xs=0, s=2-4, m=6-8, l=10. high r...",[],NaN,athleisure casual,weekend,bottom,semifitted
48990,01e2kxz0wnq96s64tdze3wb7cx,atm anthony thomas melillo,5504588,brushed twill crop wide leg pants,brushed twill gives these cropped wide-leg pan...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true to size. high rise.,[],NaN,modern businesscasual glam casual,work weekend,bottom,relaxed
48991,01e2kxwwy6jagened7p62090np,atm anthony thomas melillo,4496777,slim crop pants,"tailored from a soft, dense knit with cropped,...",unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,mid rise. true to size.,[],NaN,classic retro businesscasual,work daytonight,bottom,fittedtailored
48992,01e2kztepe7qk1v8thasf7hwnr,atm anthony thomas melillo,5289605,camo print silk skirt,flatlock-stitched bias seams enhance the fluid...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,"true to size. xs=0, s=2-4, m=6-8, l=10.",[],NaN,retro casual glam,daytonight weekend,bottom,relaxed


In [17]:
# Preparing to remove the stopwords

stopwords_gensim = list(STOPWORDS)

stopwords_NLTK = list(stopwords.words("english"))

stopwords_combined = list(set(stopwords_gensim+stopwords_NLTK)) #to remove duplicates

negatives = ['not','nor','no','neither', 'never'] #took out the negative words for a more accurate analysis
stopwords_combined = list(filter(lambda x: x not in negatives, stopwords_combined))

stopwords_combined.sort()
stopwords_expression = '|'.join(stopwords_combined)
stopwords_pattern = f'({stopwords_expression})'


In [18]:
# Defining function to stem the texts in each cell

def stem_text(text):
    porter=PorterStemmer()
    tokens = text.split()
    stemmed_tokens = [porter.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)



#source: https://stackoverflow.com/questions/43795310/apply-porters-stemmer-to-a-pandas-column-for-each-word

In [19]:
# For the columns shown below, this for loop will
# 1. remove all the punctuations and '\n' and only keep the words
# 2. remove all the stopwords defined above
# 3. perform stemming
# for each of the cells in the dataframe


# wasn't sure if we should remove brand in this list or not. Kept it for now.
col = ['brand', 'name', 'description', 'brand_category', 'details', 'labels', 'tsv']

for i in col:
    train[i] = train[i].astype(str)
    train[i] = train[i].str.replace('[^\w\s]',' ')
    train[i] = train[i].str.replace("\n", " ")
    train[i] = train[i].str.replace(rf'\b{stopwords_pattern}\b','')
    train[i] = train[i].apply(stem_text)

train

,product_id,brand,mpn,name,description,brand_category,brand_canonical_url,details,labels,tsv,style,occasion,category,fit
0,01dsrpsztdw2pgk1ywyxjgkzz0,fila,400010319073,origin fit sneaker,vintag fit leather sneaker logo print design s...,themensstor shoe sneaker lowtop,https://www.saksfifthavenue.com/fila-original-...,leather synthet upper lace closur textil line ...,need attribut,design 12 fila 1a fit 3a 6 leather 7 logo 10 m...,,,,
1,01dsqxjbx0r7dcw7ktac1sw547,chanel,400011497371,hat,nan,unknown,https://www.saksfifthavenue.com/chanel-hat/pro...,wool tweed felt,need review,chanel 1a hat 2a,,,,
2,01dpgv8tgrab993pf7z3ywg2vr,frame,lwbt0099,petit oval buckl belt,timeless leather belt craft smooth cowhid mini...,accessori,https://frame-store.com/products/petit-oval-bu...,nan,,belt 5a 9 buckl 4a 21 cowhid 13 craft 10 finis...,,,,
3,01dsr8g3f7dbrtmp8thf97xsq2,lilli pulitz kid,400011844797,littl gir girl ariana piec upf 50 swimsuit,pretti ruffl sleev trim elev essenti tank swim...,justkid girls214 girl swimwearcoverup justkid ...,https://www.saksfifthavenue.com/lilly-pulitzer...,scoopneck adjust strap flutter sleev pull styl...,need review,50 14a allov 28 ariana 9a color 27 elev 21 ess...,,,,
4,01dsr8g5gp519dedcskbmwqvk5,kissi kissi,400094474585,babi girl endear eleph pima cotton convert gown,versatil convert gown eleph appliqu,justkid baby024month infantgirl footiesromp,https://www.saksfifthavenue.com/kissy-kissy-ba...,v necklin long sleev snap closur elastic pima ...,need review,appliqu 17 babi 3a convert 10a 13 cotton 9a el...,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48989,01e2kz045bct1q2z2pbc0cwrgh,atm anthoni thoma melillo,5441202,micro twill pull pant,casual trouser cut easi drapey fit thigh taper...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size xs 0 2 4 6 8 l 10 high rise,,nan,athleisure casual,weekend,bottom,semifitted
48990,01e2kxz0wnq96s64tdze3wb7cx,atm anthoni thoma melillo,5504588,brush twill crop wide leg pant,brush twill give crop wide leg pant subtl sheen,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size high rise,,nan,modern businesscasual glam casual,work weekend,bottom,relaxed
48991,01e2kxwwy6jagened7p62090np,atm anthoni thoma melillo,4496777,slim crop pant,tailor soft dens knit crop cuf hem lean straig...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,mid rise true size,,nan,classic retro businesscasual,work daytonight,bottom,fittedtailored
48992,01e2kztepe7qk1v8thasf7hwnr,atm anthoni thoma melillo,5289605,camo print silk skirt,flatlock stitch bia seam enhanc fluid drape pu...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size xs 0 2 4 6 8 l 10,,nan,retro casual glam,daytonight weekend,bottom,relaxed


In [20]:
data = train
data = data.replace('nan',np.NaN)
data = data.replace('',np.NaN)
data_label = data.loc[(data['style'].notna()) | 
                  (data['occasion'].notna()) |
                  (data['category'].notna()) |
                  (data['fit'].notna())]
data_label.reset_index(inplace = True)

In [21]:
data

,product_id,brand,mpn,name,description,brand_category,brand_canonical_url,details,labels,tsv,style,occasion,category,fit
0,01dsrpsztdw2pgk1ywyxjgkzz0,fila,400010319073,origin fit sneaker,vintag fit leather sneaker logo print design s...,themensstor shoe sneaker lowtop,https://www.saksfifthavenue.com/fila-original-...,leather synthet upper lace closur textil line ...,need attribut,design 12 fila 1a fit 3a 6 leather 7 logo 10 m...,NaN,NaN,NaN,NaN
1,01dsqxjbx0r7dcw7ktac1sw547,chanel,400011497371,hat,NaN,unknown,https://www.saksfifthavenue.com/chanel-hat/pro...,wool tweed felt,need review,chanel 1a hat 2a,NaN,NaN,NaN,NaN
2,01dpgv8tgrab993pf7z3ywg2vr,frame,lwbt0099,petit oval buckl belt,timeless leather belt craft smooth cowhid mini...,accessori,https://frame-store.com/products/petit-oval-bu...,NaN,NaN,belt 5a 9 buckl 4a 21 cowhid 13 craft 10 finis...,NaN,NaN,NaN,NaN
3,01dsr8g3f7dbrtmp8thf97xsq2,lilli pulitz kid,400011844797,littl gir girl ariana piec upf 50 swimsuit,pretti ruffl sleev trim elev essenti tank swim...,justkid girls214 girl swimwearcoverup justkid ...,https://www.saksfifthavenue.com/lilly-pulitzer...,scoopneck adjust strap flutter sleev pull styl...,need review,50 14a allov 28 ariana 9a color 27 elev 21 ess...,NaN,NaN,NaN,NaN
4,01dsr8g5gp519dedcskbmwqvk5,kissi kissi,400094474585,babi girl endear eleph pima cotton convert gown,versatil convert gown eleph appliqu,justkid baby024month infantgirl footiesromp,https://www.saksfifthavenue.com/kissy-kissy-ba...,v necklin long sleev snap closur elastic pima ...,need review,appliqu 17 babi 3a convert 10a 13 cotton 9a el...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48989,01e2kz045bct1q2z2pbc0cwrgh,atm anthoni thoma melillo,5441202,micro twill pull pant,casual trouser cut easi drapey fit thigh taper...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size xs 0 2 4 6 8 l 10 high rise,NaN,NaN,athleisure casual,weekend,bottom,semifitted
48990,01e2kxz0wnq96s64tdze3wb7cx,atm anthoni thoma melillo,5504588,brush twill crop wide leg pant,brush twill give crop wide leg pant subtl sheen,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size high rise,NaN,NaN,modern businesscasual glam casual,work weekend,bottom,relaxed
48991,01e2kxwwy6jagened7p62090np,atm anthoni thoma melillo,4496777,slim crop pant,tailor soft dens knit crop cuf hem lean straig...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,mid rise true size,NaN,NaN,classic retro businesscasual,work daytonight,bottom,fittedtailored
48992,01e2kztepe7qk1v8thasf7hwnr,atm anthoni thoma melillo,5289605,camo print silk skirt,flatlock stitch bia seam enhanc fluid drape pu...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size xs 0 2 4 6 8 l 10,NaN,NaN,retro casual glam,daytonight weekend,bottom,relaxed


In [22]:
data_label

,index,product_id,brand,mpn,name,description,brand_category,brand_canonical_url,details,labels,tsv,style,occasion,category,fit
0,6,01dtjcerf6f4nrz2wsjffa1eys,theori,1197027,teah stretch silk camisol,beig stretch silk slip 93 silk 7 spandex dri c...,cloth top tank cami,https://www.net-a-porter.com/us/en/product/119...,fit true size normal size cut slightli loos fi...,NaN,7 15 93 13 beig 7 camisol 6a clean 18 dri 17 i...,glam businesscasual classic modern romantic ca...,weekend work daytonight nightout,top,semifitted relaxed
1,8,01dvpbj6464ykygvae0a1hmkgn,alexand wang,1202011,layer velvet mini dress,black velvet conceal hook zip fasten 65 viscos...,cloth dress mini,https://www.net-a-porter.com/us/en/product/120...,fit true size normal size design fit bust wais...,NaN,100 21 35 18 65 16 alexand 1a 15 black 7 clean...,modern classic glam,nightout weekend,onepiece,semifitted
2,10,01dpetns8db4mar29er6esa7xa,j crew,b5275,faux leather pleat midi skirt,accordion pleat flatter midi length faux leath...,skirt,https://www.jcrew.com/p/womens_category/skirts...,NaN,NaN,accordion 9 faux 3a 18 faux leath 2a 17 feel 2...,classic businesscasual modern,daytonight work,bottom,semifitted
3,11,01dt50zst2px9c1kkza4p73q5q,alexand wang,1154478,parker logo embellish snake effect leather ank...,heel measur approxim 75mm 3 inch snake effect ...,shoe boot ankl,https://www.net-a-porter.com/us/en/product/115...,fit true size normal size italian size,need attribut,3 17 75mm 16 alexand 1a 26 ankl 11a approxim 1...,edgy modern businesscasual glam,work weekend daytonight nightout,shoe,NaN
4,17,01dvcqrrx4qyjv20667zk52s82,theori,1205277,silk crepe midi dress,black silk crepe conceal zip fasten 100 silk l...,cloth dress midi,https://www.net-a-porter.com/us/en/product/120...,fit true size normal size cut slim fit drop wa...,NaN,100 16 24 35 21 65 19 14 15 black 7 clean 27 c...,retro romantic businesscasual classic modern,daytonight work nightout weekend,onepiece,semifitted relaxed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3965,48989,01e2kz045bct1q2z2pbc0cwrgh,atm anthoni thoma melillo,5441202,micro twill pull pant,casual trouser cut easi drapey fit thigh taper...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size xs 0 2 4 6 8 l 10 high rise,NaN,NaN,athleisure casual,weekend,bottom,semifitted
3966,48990,01e2kxz0wnq96s64tdze3wb7cx,atm anthoni thoma melillo,5504588,brush twill crop wide leg pant,brush twill give crop wide leg pant subtl sheen,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size high rise,NaN,NaN,modern businesscasual glam casual,work weekend,bottom,relaxed
3967,48991,01e2kxwwy6jagened7p62090np,atm anthoni thoma melillo,4496777,slim crop pant,tailor soft dens knit crop cuf hem lean straig...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,mid rise true size,NaN,NaN,classic retro businesscasual,work daytonight,bottom,fittedtailored
3968,48992,01e2kztepe7qk1v8thasf7hwnr,atm anthoni thoma melillo,5289605,camo print silk skirt,flatlock stitch bia seam enhanc fluid drape pu...,unknown,https://shop.nordstrom.com/s/atm-anthony-thoma...,true size xs 0 2 4 6 8 l 10,NaN,NaN,retro casual glam,daytonight weekend,bottom,relaxed


**I was worried about the limited size of the data with label (3,970 rows), which was less than 10% of the entire dataset (48,087 rows).**
However, when I checked the # of products that have been labeled through the sme_tag + additional_tag, there were only 3970 unique products, which matched our number of labelled products. This means that we were able to get all the labels for products that have been labelled by the SME.

In [23]:
tag['product_id'].nunique()

3970

In [24]:
data_label['product_id'].nunique()

3970